# Install necessary packages

In [1]:
!pip install tensorflow-gpu

You are using pip version 9.0.1, however version 19.1 is available.
You should consider upgrading via the 'python -m pip install --upgrade pip' command.


You are using pip version 9.0.1, however version 19.1 is available.
You should consider upgrading via the 'python -m pip install --upgrade pip' command.


## Load modules

In [1]:
import sys
sys.path.append(os.path.join('..','ext','neuron'))
import os, glob
import numpy as np
from Data_generator import data_generator
from Zip_data import Unzip_class
from Utils import normalize, visualize_model, create_model, train, load_atlas

Using TensorFlow backend.
c:\users\bmanderson\appdata\local\programs\python\python36\lib\site-packages\dicom\__init__.py:53: UserWarning: 
This code is using an older version of pydicom, which is no longer 
maintained as of Jan 2017.  You can access the new pydicom features and API 
by installing `pydicom` from PyPI.
See 'Transitioning to pydicom 1.x' section at pydicom.readthedocs.org 
for more information.

  warnings.warn(msg)


In [ ]:
%load_ext tensorboard.notebook

## Define model location and registered data
Also unzip data if we haven't already

In [2]:
data_dir = os.path.join('..','Reg_Data')
if not os.path.exists(data_dir):
    print('Unzipping data')
    Unzip_class(os.path.join('..','Reg_Data'),r'..')
    print('Finished unzipping')
model_dir = os.path.join('..','models')

## Load in the atlas file

In [3]:
atlas_file= os.path.join('..','reg_data','Atlas_Data.npy')
atlas_vol = load_atlas(atlas_file)

### Creating your model

This is the part that you have the most unique control over, what exactly will your model look like?

One good way of visualizing the model is to create it and look at the graphical architecture with tensorboard

Lets start off with something simple, a UNet with one concatentation layer

In [4]:
layers = {'Layer_0':{'Encoding':[16,32],'Decoding':[32,16,8]},
          'Base':{'Encoding':[64]}}
model_desc = 'Shallow_net' # Name of your model
# The numbers inside are the number of filter banks, you can have mulitple filter banks per layer

This should be imagined as a U net, the numbers within [] are the number of filters in the convolution blocks, the network will first perform convolutions downthe encoding side to the base, and then concatenate from the previous encoding layer before performing decoding convolutions

This might not look like much, but lets see what it looks like graphically

## Visualize the model

In [5]:
visualize_model(layers, atlas_vol.shape[1:-1], model_desc)

Model created at: K:\Morfeus\AAPM_SummerSchool\bma_voxel_morph\Tensorboard_models\Shallow_net


In [ ]:
logs_base_dir = os.path.join('..','Tensorboard_models')
%tensorboard --logdir {logs_base_dir}

## Load training data

In [6]:
train_generator = data_generator(atlas_vol,data_dir)
print('We have ' + str(len(train_generator)) + ' registrations available')

We have 116 registrations available


## Define hyperparameters
These are variables which you will manipulate in order to create the best model possible

In [7]:
learning_rate = 0.001 # Rate at which our gradients will change during each back propogation, typically in range of 1e-2 to 1e-5
number_of_epochs = 10 # The number of epochs to be trained, one epoch means that you have seen the entirety of your dataset
                      # However, since we defined steps per epoch this might not apply
regularization_parameter = 0.01 # Lambda in regularization equation
steps_per_epoch = 30
loss_function = 'mse'
batch_normalization = True

## Create the model

In [8]:
model, callbacks = create_model(layers, atlas_vol.shape[1:-1], model_desc, batch_norm=batch_normalization)

In [10]:
train(model, train_generator, callbacks, learning_rate, number_of_epochs, 
      regularization_parameter, steps_per_epoch,loss_function)

Epoch 1/10
30/30 [==============================] - ETA: 10:08 - loss: 0.0227 - spatial_transformer_1_loss: 0.0227 - flow_loss: 1.5133e-0 - ETA: 5:18 - loss: 0.0242 - spatial_transformer_1_loss: 0.0242 - flow_loss: 1.1124e-0 - ETA: 3:40 - loss: 0.0229 - spatial_transformer_1_loss: 0.0229 - flow_loss: 3.7514e- - ETA: 2:50 - loss: 0.0254 - spatial_transformer_1_loss: 0.0253 - flow_loss: 8.9111e- - ETA: 2:19 - loss: 0.0233 - spatial_transformer_1_loss: 0.0233 - flow_loss: 0.0018   - ETA: 1:58 - loss: 0.0247 - spatial_transformer_1_loss: 0.0246 - flow_loss: 0.00 - ETA: 1:43 - loss: 0.0240 - spatial_transformer_1_loss: 0.0240 - flow_loss: 0.00 - ETA: 1:31 - loss: 0.0245 - spatial_transformer_1_loss: 0.0244 - flow_loss: 0.00 - ETA: 1:21 - loss: 0.0235 - spatial_transformer_1_loss: 0.0234 - flow_loss: 0.00 - ETA: 1:13 - loss: 0.0231 - spatial_transformer_1_loss: 0.0230 - flow_loss: 0.00 - ETA: 1:06 - loss: 0.0225 - spatial_transformer_1_loss: 0.0223 - flow_loss: 0.01 - ETA: 1:00 - loss: 0.023

OSError: Unable to create file (unable to open file: name = '..\models\Shallow_net\Model_saves\weights-improvement-01.hdf5', errno = 2, error message = 'No such file or directory', flags = 13, o_flags = 302)

Play around! Add more layers, convolutions, see if you can reduce your loss the most, change the bottom two files, make sure to rename your model

In [14]:
layers = {'Layer_0':{'Encoding':[16],'Decoding':[16,16]},
          'Layer_1':{'Encoding':[32],'Decoding':[32,32]},
          'Layer_2':{'Encoding':[32],'Decoding':[32]},
          'Layer_3':{'Encoding':[32],'Decoding':[32]},
          'Base':{'Encoding':[32]}}
model_desc = 'Deep_net' # Name of your model
# The numbers inside are the number of filter banks, you can have mulitple filter banks per layer
visualize_model(layers, atlas_vol.shape[1:-1], model_desc)
logs_base_dir = os.path.join('..','Tensorboard_models')
%tensorboard --logdir {logs_base_dir}

In [5]:
visualize_model(layers, atlas_vol.shape[1:-1], model_desc)

Model created at: K:\Morfeus\AAPM_SummerSchool\bma_voxel_morph\Tensorboard_models\Shallow_net


View your model by scrolling up to look at the tensorboard module

In [15]:
model, callbacks = create_model(layers, atlas_vol.shape[1:-1], model_desc)

K:\Morfeus\AAPM_SummerSchool\bma_voxel_morph\Tensorboard_models\Deep_net


In [ ]:
train(model, train_generator, callbacks, learning_rate, number_of_epochs, 
      regularization_parameter, steps_per_epoch,loss_function)